<a href="https://colab.research.google.com/github/snbigft/api_rest/blob/main/Api_Rest_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fastapi uvicorn psycopg2-binary nest-asyncio pyngrok



In [ ]:
import json
import psycopg2
import nest_asyncio
import uvicorn
from fastapi import FastAPI, Query
from pyngrok import ngrok

# Aplicar nest_asyncio para evitar conflictos de bucle de eventos en Colab
nest_asyncio.apply()

# Configurar ngrok con tu authtoken
ngrok.set_auth_token("YOUR_NGROK_AUTHTOKEN")

# Configuración de la base de datos
DB_HOST = "ep-snowy-dawn-a8omlpke-pooler.eastus2.azure.neon.tech"
DB_NAME = "neondb"
DB_USER = "neondb_owner"
DB_PASSWORD = "npg_BcpnId62FYhS"

# Función para conectar a PostgreSQL
def get_db_connection():
    return psycopg2.connect(
        host=DB_HOST,
        dbname=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD,
        sslmode="require"
    )

# Crear la API con FastAPI
app = FastAPI()

@app.get("/customers")
def get_customer_data(query: str = Query(..., description="Valor a buscar en customer_data")):
    try:
        conn = get_db_connection()
        cursor = conn.cursor()

        # Ejecutar consulta con parámetro seguro
        cursor.execute("SELECT customer_data FROM CUSTOMERS WHERE customer_data::text ILIKE %s", (f"%{query}%",))
        rows = cursor.fetchall()

        # Devolver los datos directamente sin intentar cargarlos como JSON
        result = [row[0] for row in rows]

        cursor.close()
        conn.close()

        return {"results": result}

    except Exception as e:
        return {"error": str(e)}

# Exponer la API con ngrok en Google Colab
public_url = ngrok.connect(8000).public_url
print(f"🔥 API en ejecución en: {public_url}/customers?query=imone")

# Iniciar el servidor uvicorn sin llamar directamente a asyncio.run()
uvicorn.run(app, host="0.0.0.0", port=8000)


🔥 API en ejecución en: https://b896-35-223-217-142.ngrok-free.app/docs


/usr/local/lib/python3.11/dist-packages/uvicorn/protocols/websockets/auto.py:19: RuntimeWarning: coroutine 'Server.serve' was never awaited
  from uvicorn.protocols.websockets.websockets_impl import WebSocketProtocol
INFO:     Started server process [817]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     139.47.125.21:0 - "GET /customers?query=imone HTTP/1.1" 200 OK
INFO:     139.47.125.21:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     139.47.125.21:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     139.47.125.21:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     139.47.125.21:0 - "GET /customers?query=imone HTTP/1.1" 200 OK
INFO:     139.47.125.21:0 - "GET /customers?query=677 HTTP/1.1" 200 OK
INFO:     139.47.125.21:0 - "GET /customers?query=s HTTP/1.1" 200 OK
INFO:     139.47.125.21:0 - "GET /customers?query=mascota HTTP/1.1" 200 OK
INFO:     139.47.125.21:0 - "GET /customers?query=apellido HTTP/1.1" 200 OK
